# Iowa (USA) Liquor Sales 2018-2022 Cleaned | Transformed

In [1]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import datetime as dt
from math import sqrt, pow
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# warnings
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

%matplotlib inline

In [2]:
df = pd.read_csv("data/fact_Sales.csv", low_memory=False)

### Format/Convert data

In [3]:
# convert to datetime type

df['date'] = pd.to_datetime(df['date'])

In [4]:
# extract day, month, year from date

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['dayofweek'] = df['date'].dt.day_name()

In [5]:
# Get type of order/returned (RINV)

invoice_type = df['invoice_line_no'].str.split(pat = '-', expand = True)

In [6]:
df['invoice_type'] = invoice_type[0]

In [7]:
# Format displayed years: 2018-2022

order_of_years = CategoricalDtype(categories=["2018", "2019", "2020", "2021", "2022"], ordered=True)

In [8]:
df['year'] = df['year'].astype(str)
df['month'] = df['month'].astype(str)
df['day'] = df['day'].astype(str)

#city_type = CategoricalDtype(categories=["A", "B", "C"], ordered=True)<br>
#city_cat_col = df['City_Category'].astype(city_type).value_counts(sort=False)

In [9]:
df.head(5)

,invoice_line_no,date,sale_bottles,sale_dollars,sale_liters,sale_gallons,store_ID,city_ID,county_ID,itemno,item_group,category_ID,vendor_ID,year,month,day,dayofweek,invoice_type
0,INV-54501400024,2022-12-30,24,66.96,9.00,2.37,0,243,30,36304,0,9.0,177.0,2022,12,30,Friday,INV
1,INV-54509700008,2022-12-30,6,72.00,4.50,1.18,1,887,24,36969,1,9.0,98.0,2022,12,30,Friday,INV
2,INV-54528400012,2022-12-30,12,198.00,9.00,2.37,2,527,56,77776,2,3.0,44.0,2022,12,30,Friday,INV
3,INV-54523600136,2022-12-30,1,19.20,0.05,0.01,3,112,28,38194,3,9.0,106.0,2022,12,30,Friday,INV
4,INV-54519400027,2022-12-30,12,180.00,9.00,2.37,4,782,35,38176,4,9.0,106.0,2022,12,30,Friday,INV


# 2018-2022

### Total revenue by years

In [10]:
revenue_byYear_sum = df.groupby(['year', 'invoice_type'])['sale_dollars'].sum()

df_revenue_byYear_sum = revenue_byYear_sum.apply(lambda x: f'{x:,.1f}').to_frame()
df_revenue_byYear_sum

sale_dollars
year invoice_type               
2018 INV           334,246,947.4
2019 INV           349,251,975.2
2020 INV           396,701,312.5
2021 INV           428,168,786.7
2022 INV           429,020,335.1
     RINV             -184,893.4

### Number of orders by years

In [11]:
numOfOrders_byYear_count = df.groupby('year')['invoice_type'].value_counts()

df_numOfOrders_byYear_count = numOfOrders_byYear_count.map('{:,d}'.format).to_frame(name='count')
df_numOfOrders_byYear_count

count
year invoice_type           
2018 INV           2,355,711
2019 INV           2,380,532
2020 INV           2,614,587
2021 INV           2,623,471
2022 INV           2,564,360
     RINV              1,085

### Bottle by years

In [12]:
bottle_byYear_sum = df.groupby(['year', 'invoice_type'])['sale_bottles'].sum()

df_bottle_byYear_sum = bottle_byYear_sum.map('{:,d}'.format).to_frame(name='count')
df_bottle_byYear_sum

count
year invoice_type            
2018 INV           25,437,840
2019 INV           26,847,391
2020 INV           29,844,881
2021 INV           31,207,340
2022 INV           30,448,795
     RINV             -11,709

### Best store 

In [13]:
df_store = pd.read_csv("data/dim_Stores.csv")
df_city = pd.read_csv("data/dim_City.csv")
df_county = pd.read_csv("data/dim_County.csv")
df_item = pd.read_csv("data/dim_Items.csv")
df_vendor = pd.read_csv("data/dim_Vendor.csv")
df_cat = pd.read_csv("data/dim_Category.csv")

In [14]:
best_store_2018 = df.query("year == '2018'").groupby(['invoice_type'])['store_ID'].value_counts().nlargest(10)

df_best_store_2018 = best_store_2018.map('{:,d}'.format).to_frame(name='2018_invoices')

df_best_store_2018_merge_store = df_best_store_2018.merge(df_store, on=['store_ID'], how = 'left')
df_best_store_2018_merge_store_dropcol = df_best_store_2018_merge_store.drop(['store','brand_store_no',
                                                                              'store_location','county_ID'], axis =1)

df_best_store_2018_merge_city = df_best_store_2018_merge_store_dropcol.merge(df_city, on=['city_ID'], how = 'left')
df_best_store_2018_merge_city_dropcol = df_best_store_2018_merge_city.drop(['city_ID'], axis =1)

df_best_store_2018_merge_county = df_best_store_2018_merge_city_dropcol.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_store_2018 = df_best_store_2018_merge_county.drop(['county_ID'], axis =1)

final_df_best_store_2018

,store_ID,2018_invoices,name,address,zipcode,city,county
0,344,"28,538",HY-VEE,3221 SE 14TH ST,50320.0,DES MOINES,POLK
1,291,"17,771",CENTRAL CITY,1501 MICHIGAN AVE,50314.0,DES MOINES,POLK
2,407,"15,694",HY-VEE FOOD STORE,6301 UNIVERSITY,50613.0,CEDAR FALLS,BLACK HAWK
3,263,"13,223",CENTRAL CITY LIQUOR INC.,1460 2ND AVE,50314.0,DES MOINES,POLK
4,790,"13,016",HY-VEE FOOD STORE,2004 8TH ST,52241.0,CORALVILLE,JOHNSON
5,1037,"12,898",HY-VEE WINE AND SPIRITS,2890 DEVILS GLEN RD,52722.0,BETTENDORF,SCOTT
6,410,"12,280",HY-VEE FOOD STORE,3800 W LINCOLN WAY,50010.0,AMES,STORY
7,777,"12,223",HY-VEE WINE AND SPIRITS,1720 WATERFRONT DR,52240.0,IOWA CITY,JOHNSON
8,354,"11,931",CYCLONE LIQUORS,626 LINCOLN WAY,50010.0,AMES,STORY
9,374,"11,880",HY-VEE,555 S 51ST ST,50265.0,WEST DES MOINES,POLK


In [15]:
best_store_2019 = df.query("year == '2019'").groupby(['invoice_type'])['store_ID'].value_counts().nlargest(10)

df_best_store_2019 = best_store_2019.map('{:,d}'.format).to_frame(name='2019_invoices')

df_best_store_2019_merge_store = df_best_store_2019.merge(df_store, on=['store_ID'], how = 'left')
df_best_store_2019_merge_store_dropcol = df_best_store_2019_merge_store.drop(['store','brand_store_no',
                                                                              'store_location','county_ID'], axis =1)

df_best_store_2019_merge_city = df_best_store_2019_merge_store_dropcol.merge(df_city, on=['city_ID'], how = 'left')
df_best_store_2019_merge_city_dropcol = df_best_store_2019_merge_city.drop(['city_ID'], axis =1)

df_best_store_2019_merge_county = df_best_store_2019_merge_city_dropcol.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_store_2019 = df_best_store_2019_merge_county.drop(['county_ID'], axis =1)

final_df_best_store_2019

,store_ID,2019_invoices,name,address,zipcode,city,county
0,344,"29,433",HY-VEE,3221 SE 14TH ST,50320.0,DES MOINES,POLK
1,291,"17,816",CENTRAL CITY,1501 MICHIGAN AVE,50314.0,DES MOINES,POLK
2,263,"15,706",CENTRAL CITY LIQUOR INC.,1460 2ND AVE,50314.0,DES MOINES,POLK
3,407,"15,044",HY-VEE FOOD STORE,6301 UNIVERSITY,50613.0,CEDAR FALLS,BLACK HAWK
4,777,"13,503",HY-VEE WINE AND SPIRITS,1720 WATERFRONT DR,52240.0,IOWA CITY,JOHNSON
5,1037,"13,167",HY-VEE WINE AND SPIRITS,2890 DEVILS GLEN RD,52722.0,BETTENDORF,SCOTT
6,379,"12,141",HY-VEE FOOD STORE,2400 4TH ST SW,50401.0,MASON CITY,CERRO GORDO
7,790,"11,966",HY-VEE FOOD STORE,2004 8TH ST,52241.0,CORALVILLE,JOHNSON
8,354,"11,908",CYCLONE LIQUORS,626 LINCOLN WAY,50010.0,AMES,STORY
9,374,"11,216",HY-VEE,555 S 51ST ST,50265.0,WEST DES MOINES,POLK


In [16]:
best_store_2020 = df.query("year == '2020'").groupby(['invoice_type'])['store_ID'].value_counts().nlargest(10)

df_best_store_2020 = best_store_2020.map('{:,d}'.format).to_frame(name='2020_invoices')

df_best_store_2020_merge_store = df_best_store_2020.merge(df_store, on=['store_ID'], how = 'left')
df_best_store_2020_merge_store_dropcol = df_best_store_2020_merge_store.drop(['store','brand_store_no',
                                                                              'store_location','county_ID'], axis =1)

df_best_store_2020_merge_city = df_best_store_2020_merge_store_dropcol.merge(df_city, on=['city_ID'], how = 'left')
df_best_store_2020_merge_city_dropcol = df_best_store_2020_merge_city.drop(['city_ID'], axis =1)

df_best_store_2020_merge_county = df_best_store_2020_merge_city_dropcol.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_store_2020 = df_best_store_2020_merge_county.drop(['county_ID'], axis =1)

final_df_best_store_2020

,store_ID,2020_invoices,name,address,zipcode,city,county
0,344,"20,834",HY-VEE,3221 SE 14TH ST,50320.0,DES MOINES,POLK
1,263,"17,241",CENTRAL CITY LIQUOR INC.,1460 2ND AVE,50314.0,DES MOINES,POLK
2,407,"15,375",HY-VEE FOOD STORE,6301 UNIVERSITY,50613.0,CEDAR FALLS,BLACK HAWK
3,800,"14,313",HY-VEE,5050 EDGEWOOD RD,52411.0,CEDAR RAPIDS,LINN
4,1048,"13,874",HY-VEE FOOD STORE,2510 SW STATE ST,50023.0,ANKENY,POLK
5,1037,"13,616",HY-VEE WINE AND SPIRITS,2890 DEVILS GLEN RD,52722.0,BETTENDORF,SCOTT
6,291,"13,386",CENTRAL CITY,1501 MICHIGAN AVE,50314.0,DES MOINES,POLK
7,379,"13,046",HY-VEE FOOD STORE,2400 4TH ST SW,50401.0,MASON CITY,CERRO GORDO
8,374,"12,890",HY-VEE,555 S 51ST ST,50265.0,WEST DES MOINES,POLK
9,778,"11,976",HY-VEE,1823 E KIMBERLY RD,52807.0,DAVENPORT,SCOTT


In [17]:
best_store_2021 = df.query("year == '2021'").groupby(['invoice_type'])['store_ID'].value_counts().nlargest(10)

df_best_store_2021 = best_store_2021.to_frame(name='2021_invoices')

df_best_store_2021_merge_store = df_best_store_2021.merge(df_store, on=['store_ID'], how = 'left')
df_best_store_2021_merge_store_dropcol = df_best_store_2021_merge_store.drop(['store','brand_store_no',
                                                                              'store_location','county_ID'], axis =1)

df_best_store_2021_merge_city = df_best_store_2021_merge_store_dropcol.merge(df_city, on=['city_ID'], how = 'left')
df_best_store_2021_merge_city_dropcol = df_best_store_2021_merge_city.drop(['city_ID'], axis =1)

df_best_store_2021_merge_county = df_best_store_2021_merge_city_dropcol.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_store_2021 = df_best_store_2021_merge_county.drop(['county_ID'], axis =1)

final_df_best_store_2021

,store_ID,2021_invoices,name,address,zipcode,city,county
0,344,25832,HY-VEE,3221 SE 14TH ST,50320.0,DES MOINES,POLK
1,291,18357,CENTRAL CITY,1501 MICHIGAN AVE,50314.0,DES MOINES,POLK
2,263,15825,CENTRAL CITY LIQUOR INC.,1460 2ND AVE,50314.0,DES MOINES,POLK
3,407,14549,HY-VEE FOOD STORE,6301 UNIVERSITY,50613.0,CEDAR FALLS,BLACK HAWK
4,779,13611,BENZ DISTRIBUTING,501 7TH AVE SE,52401.0,CEDAR RAPIDS,LINN
5,800,13420,HY-VEE,5050 EDGEWOOD RD,52411.0,CEDAR RAPIDS,LINN
6,380,13146,HAPPY'S WINE & SPIRITS,5925 UNIVERSITY AVE,50613.0,CEDAR FALLS,BLACK HAWK
7,1048,12812,HY-VEE FOOD STORE,2510 SW STATE ST,50023.0,ANKENY,POLK
8,790,12592,HY-VEE FOOD STORE,2004 8TH ST,52241.0,CORALVILLE,JOHNSON
9,1037,12553,HY-VEE WINE AND SPIRITS,2890 DEVILS GLEN RD,52722.0,BETTENDORF,SCOTT


In [18]:
best_store_2022 = df.query("year == '2022' & invoice_type == 'INV'").groupby(['invoice_type'])['store_ID'].value_counts().nlargest(10)

df_best_store_2022 = best_store_2022.map('{:,d}'.format).to_frame(name='2022_invoices')

df_best_store_2022_merge_store = df_best_store_2022.merge(df_store, on=['store_ID'], how = 'left')
df_best_store_2022_merge_store_dropcol = df_best_store_2022_merge_store.drop(['store','brand_store_no',
                                                                              'store_location','county_ID'], axis =1)

df_best_store_2022_merge_city = df_best_store_2022_merge_store_dropcol.merge(df_city, on=['city_ID'], how = 'left')
df_best_store_2022_merge_city_dropcol = df_best_store_2022_merge_city.drop(['city_ID'], axis =1)

df_best_store_2022_merge_county = df_best_store_2022_merge_city_dropcol.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_store_2022 = df_best_store_2022_merge_county.drop(['county_ID'], axis =1)

final_df_best_store_2022

,store_ID,2022_invoices,name,address,zipcode,city,county
0,344,"25,981",HY-VEE,3221 SE 14TH ST,50320.0,DES MOINES,POLK
1,291,"18,807",CENTRAL CITY,1501 MICHIGAN AVE,50314.0,DES MOINES,POLK
2,800,"14,454",HY-VEE,5050 EDGEWOOD RD,52411.0,CEDAR RAPIDS,LINN
3,407,"13,905",HY-VEE FOOD STORE,6301 UNIVERSITY,50613.0,CEDAR FALLS,BLACK HAWK
4,779,"13,738",BENZ DISTRIBUTING,501 7TH AVE SE,52401.0,CEDAR RAPIDS,LINN
5,1037,"13,479",HY-VEE WINE AND SPIRITS,2890 DEVILS GLEN RD,52722.0,BETTENDORF,SCOTT
6,263,"13,132",CENTRAL CITY LIQUOR INC.,1460 2ND AVE,50314.0,DES MOINES,POLK
7,380,"12,908",HAPPY'S WINE & SPIRITS,5925 UNIVERSITY AVE,50613.0,CEDAR FALLS,BLACK HAWK
8,790,"11,515",HY-VEE FOOD STORE,2004 8TH ST,52241.0,CORALVILLE,JOHNSON
9,423,"10,917",HY-VEE FOOD STORE,1422 FLAMMANG DR,50702.0,WATERLOO,BLACK HAWK


In [19]:
best_rstore_2022 = df.query("year == '2022' & invoice_type == 'RINV'").groupby(['invoice_type'])['store_ID'].value_counts().nlargest(10)

df_best_rstore_2022 = best_rstore_2022.to_frame(name='2022_Rinvoices')

df_best_rstore_2022_merge_store = df_best_rstore_2022.merge(df_store, on=['store_ID'], how = 'left')
df_best_rstore_2022_merge_store_dropcol = df_best_rstore_2022_merge_store.drop(['store','brand_store_no',
                                                                              'store_location','county_ID'], axis =1)

df_best_rstore_2022_merge_city = df_best_rstore_2022_merge_store_dropcol.merge(df_city, on=['city_ID'], how = 'left')
df_best_rstore_2022_merge_city_dropcol = df_best_rstore_2022_merge_city.drop(['city_ID'], axis =1)

df_best_rstore_2022_merge_county = df_best_rstore_2022_merge_city_dropcol.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_rstore_2022 = df_best_rstore_2022_merge_county.drop(['county_ID'], axis =1)

final_df_best_rstore_2022

,store_ID,2022_Rinvoices,name,address,zipcode,city,county
0,78,26,ANOTHER ROUND,622 S 6TH AVE,52742.0,DE WITT,CLINTON
1,344,22,HY-VEE,3221 SE 14TH ST,50320.0,DES MOINES,POLK
2,437,20,WILKIE LIQUORS,724 1ST ST NE,52314.0,MOUNT VERNON,LINN
3,380,13,HAPPY'S WINE & SPIRITS,5925 UNIVERSITY AVE,50613.0,CEDAR FALLS,BLACK HAWK
4,456,12,HY-VEE FOOD AND DRUG,4035 MT VERNON RD SE,52403.0,CEDAR RAPIDS,LINN
5,863,12,HY-VEE FOOD STORE,2400 2ND AVE,52761.0,MUSCATINE,MUSCATINE
6,1050,11,HY-VEE FOOD STORE,802 S CENTER,50158.0,MARSHALLTOWN,MARSHALL
7,1064,11,HY-VEE FOOD STORE,910 N JEFFERSON,50125.0,INDIANOLA,WARREN
8,436,10,NORTHSIDE LIQUOR AND GROCERY,1303 N FEDERAL,50401.0,MASON CITY,CERRO GORDO
9,643,9,HY-VEE WINE AND SPIRITS,2126 KIMBALL AVE,50701.0,WATERLOO,BLACK HAWK


In [89]:
# Search by Store ID

search_invoices_byStoreID = df.query("store_ID == 379").groupby(['invoice_type'])['year'].value_counts()
search_invoices_byStoreID = search_invoices_byStoreID.map('{:,d}'.format).to_frame(name='count')
search_invoices_byStoreID

count
invoice_type year        
INV          2020  13,046
             2019  12,141
             2018  11,217
             2022  10,615
             2021  10,524
RINV         2022       1

### Best City

In [24]:
best_city_2018 = df.query("year == '2018'").groupby(['invoice_type'])['city_ID'].value_counts().nlargest(10)

df_best_city_2018 = best_city_2018.map('{:,d}'.format).to_frame(name='count')

df_best_city_2018_merge_city = df_best_city_2018.merge(df_city, on=['city_ID'], how = 'left')
df_best_city_2018_merge_county = df_best_city_2018_merge_city.merge(df_county, on=['county_ID'], how = 'left')

final_df_best_city_2018 = df_best_city_2018_merge_county.drop(['county_ID'], axis =1)
final_df_best_city_2018

,city_ID,count,city,county
0,226,"202,081",DES MOINES,POLK
1,135,"154,287",CEDAR RAPIDS,LINN
2,206,"100,555",DAVENPORT,SCOTT
3,883,"79,329",WATERLOO,BLACK HAWK
4,189,"72,293",COUNCIL BLUFFS,POTTAWATTAMIE
5,786,"72,065",SIOUX CITY,WOODBURY
6,408,"67,757",IOWA CITY,JOHNSON
7,24,"66,700",AMES,STORY
8,243,"66,097",DUBUQUE,DUBUQUE
9,903,"64,625",WEST DES MOINES,POLK


In [25]:
best_city_2019 = df.query("year == '2019'").groupby(['invoice_type'])['city_ID'].value_counts().nlargest(10)

df_best_city_2019 = best_city_2019.map('{:,d}'.format).to_frame(name='count')

df_best_city_2019_merge_city = df_best_city_2019.merge(df_city, on=['city_ID'], how = 'left')
df_best_city_2019_merge_county = df_best_city_2019_merge_city.merge(df_county, on=['county_ID'], how = 'left')

final_df_best_city_2019 = df_best_city_2019_merge_county.drop(['county_ID'], axis =1)
final_df_best_city_2019

,city_ID,count,city,county
0,226,"203,534",DES MOINES,POLK
1,135,"157,195",CEDAR RAPIDS,LINN
2,206,"101,324",DAVENPORT,SCOTT
3,883,"77,336",WATERLOO,BLACK HAWK
4,786,"74,234",SIOUX CITY,WOODBURY
5,189,"70,794",COUNCIL BLUFFS,POTTAWATTAMIE
6,408,"66,704",IOWA CITY,JOHNSON
7,903,"66,701",WEST DES MOINES,POLK
8,243,"65,691",DUBUQUE,DUBUQUE
9,24,"63,157",AMES,STORY


In [26]:
best_city_2020 = df.query("year == '2020'").groupby(['invoice_type'])['city_ID'].value_counts().nlargest(10)

df_best_city_2020 = best_city_2020.map('{:,d}'.format).to_frame(name='count')

df_best_city_2020_merge_city = df_best_city_2020.merge(df_city, on=['city_ID'], how = 'left')
df_best_city_2020_merge_county = df_best_city_2020_merge_city.merge(df_county, on=['county_ID'], how = 'left')

final_df_best_city_2020 = df_best_city_2020_merge_county.drop(['county_ID'], axis =1)
final_df_best_city_2020

,city_ID,count,city,county
0,226,"209,827",DES MOINES,POLK
1,135,"169,076",CEDAR RAPIDS,LINN
2,206,"110,057",DAVENPORT,SCOTT
3,903,"81,658",WEST DES MOINES,POLK
4,883,"81,033",WATERLOO,BLACK HAWK
5,786,"77,573",SIOUX CITY,WOODBURY
6,189,"75,661",COUNCIL BLUFFS,POTTAWATTAMIE
7,24,"66,315",AMES,STORY
8,408,"65,326",IOWA CITY,JOHNSON
9,243,"61,299",DUBUQUE,DUBUQUE


In [27]:
best_city_2021 = df.query("year == '2021'").groupby(['invoice_type'])['city_ID'].value_counts().nlargest(10)

df_best_city_2021 = best_city_2021.map('{:,d}'.format).to_frame(name='count')

df_best_city_2021_merge_city = df_best_city_2021.merge(df_city, on=['city_ID'], how = 'left')
df_best_city_2021_merge_county = df_best_city_2021_merge_city.merge(df_county, on=['county_ID'], how = 'left')

final_df_best_city_2021 = df_best_city_2021_merge_county.drop(['county_ID'], axis =1)
final_df_best_city_2021

,city_ID,count,city,county
0,226,"210,648",DES MOINES,POLK
1,135,"163,295",CEDAR RAPIDS,LINN
2,206,"106,420",DAVENPORT,SCOTT
3,903,"87,719",WEST DES MOINES,POLK
4,786,"78,933",SIOUX CITY,WOODBURY
5,189,"75,147",COUNCIL BLUFFS,POTTAWATTAMIE
6,883,"74,283",WATERLOO,BLACK HAWK
7,24,"65,511",AMES,STORY
8,243,"63,906",DUBUQUE,DUBUQUE
9,408,"59,224",IOWA CITY,JOHNSON


In [28]:
best_city_2022 = df.query("year == '2022' & invoice_type == 'INV'").groupby(['invoice_type'])['city_ID'].value_counts().nlargest(10)

df_best_city_2022 = best_city_2022.map('{:,d}'.format).to_frame(name='count')

df_best_city_2022_merge_city = df_best_city_2022.merge(df_city, on=['city_ID'], how = 'left')
df_best_city_2022_merge_county = df_best_city_2022_merge_city.merge(df_county, on=['county_ID'], how = 'left')

final_df_best_city_2022 = df_best_city_2022_merge_county.drop(['county_ID'], axis =1)
final_df_best_city_2022

,city_ID,count,city,county
0,226,"212,703",DES MOINES,POLK
1,135,"165,109",CEDAR RAPIDS,LINN
2,206,"102,117",DAVENPORT,SCOTT
3,903,"81,218",WEST DES MOINES,POLK
4,883,"72,557",WATERLOO,BLACK HAWK
5,786,"72,201",SIOUX CITY,WOODBURY
6,189,"71,865",COUNCIL BLUFFS,POTTAWATTAMIE
7,24,"62,754",AMES,STORY
8,29,"61,305",ANKENY,POLK
9,408,"57,284",IOWA CITY,JOHNSON


In [29]:
best_rcity_2022 = df.query("year == '2022' & invoice_type == 'RINV'").groupby(['invoice_type'])['city_ID'].value_counts().nlargest(10)

df_best_rcity_2022 = best_rcity_2022.to_frame(name='count')

df_best_rcity_2022_merge_city = df_best_rcity_2022.merge(df_city, on=['city_ID'], how = 'left')
df_best_rcity_2022_merge_county = df_best_rcity_2022_merge_city.merge(df_county, on=['county_ID'], how = 'left')

final_df_best_rcity_2022 = df_best_rcity_2022_merge_county.drop(['county_ID'], axis =1)
final_df_best_rcity_2022

,city_ID,count,city,county
0,226,82,DES MOINES,POLK
1,135,59,CEDAR RAPIDS,LINN
2,206,49,DAVENPORT,SCOTT
3,883,35,WATERLOO,BLACK HAWK
4,903,32,WEST DES MOINES,POLK
5,211,31,DE WITT,CLINTON
6,786,24,SIOUX CITY,WOODBURY
7,593,23,MOUNT VERNON,LINN
8,596,22,MUSCATINE,MUSCATINE
9,134,21,CEDAR FALLS,BLACK HAWK


In [90]:
# Search by City ID

search_invoices_byCityID = df.query("city_ID == 226").groupby(['invoice_type'])['year'].value_counts()
search_invoices_byCityID = search_invoices_byCityID.map('{:,d}'.format).to_frame(name='count')
search_invoices_byCityID

count
invoice_type year         
INV          2022  212,703
             2021  210,648
             2020  209,827
             2019  203,534
             2018  202,081
RINV         2022       82

### County

In [30]:
best_county_2018 = df.query("year == '2018'").groupby(['invoice_type'])['county_ID'].value_counts().nlargest(10)

df_best_county_2018 = best_county_2018.map('{:,d}'.format).to_frame(name='count')

final_df_best_county_2018 = df_best_county_2018.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_county_2018

,county_ID,count,county
0,76,"432,979",POLK
1,56,"195,416",LINN
2,81,"144,693",SCOTT
3,6,"139,277",BLACK HAWK
4,51,"124,803",JOHNSON
5,77,"79,322",POTTAWATTAMIE
6,84,"77,569",STORY
7,96,"77,233",WOODBURY
8,30,"74,898",DUBUQUE
9,16,"51,174",CERRO GORDO


In [31]:
best_county_2019 = df.query("year == '2019'").groupby(['invoice_type'])['county_ID'].value_counts().nlargest(10)

df_best_county_2019 = best_county_2019.map('{:,d}'.format).to_frame(name='count')

final_df_best_county_2019 = df_best_county_2019.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_county_2019

,county_ID,count,county
0,76,"440,234",POLK
1,56,"200,027",LINN
2,81,"145,956",SCOTT
3,6,"138,576",BLACK HAWK
4,51,"124,278",JOHNSON
5,96,"80,519",WOODBURY
6,77,"78,404",POTTAWATTAMIE
7,84,"74,154",STORY
8,30,"73,890",DUBUQUE
9,16,"52,548",CERRO GORDO


In [32]:
best_county_2020 = df.query("year == '2020'").groupby(['invoice_type'])['county_ID'].value_counts().nlargest(10)

df_best_county_2020 = best_county_2020.map('{:,d}'.format).to_frame(name='count')

final_df_best_county_2020 = df_best_county_2020.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_county_2020

,county_ID,count,county
0,76,"494,816",POLK
1,56,"218,488",LINN
2,81,"157,856",SCOTT
3,6,"150,003",BLACK HAWK
4,51,"132,447",JOHNSON
5,77,"84,503",POTTAWATTAMIE
6,96,"84,211",WOODBURY
7,84,"78,802",STORY
8,30,"70,168",DUBUQUE
9,16,"53,733",CERRO GORDO


In [33]:
best_county_2021 = df.query("year == '2021'").groupby(['invoice_type'])['county_ID'].value_counts().nlargest(10)

df_best_county_2021 = best_county_2021.map('{:,d}'.format).to_frame(name='count')

final_df_best_county_2021 = df_best_county_2021.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_county_2021

,county_ID,count,county
0,76,"501,864",POLK
1,56,"215,009",LINN
2,81,"151,408",SCOTT
3,6,"146,387",BLACK HAWK
4,51,"123,166",JOHNSON
5,96,"86,407",WOODBURY
6,77,"83,762",POTTAWATTAMIE
7,84,"78,833",STORY
8,30,"74,031",DUBUQUE
9,16,"53,461",CERRO GORDO


In [34]:
best_county_2022 = df.query("year == '2022' & invoice_type == 'INV'").groupby(['invoice_type'])['county_ID'].value_counts().nlargest(10)

df_best_county_2022 = best_county_2022.map('{:,d}'.format).to_frame(name='count')

final_df_best_county_2022 = df_best_county_2022.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_county_2022

,county_ID,count,county
0,76,"501,984",POLK
1,56,"215,381",LINN
2,81,"145,111",SCOTT
3,6,"140,485",BLACK HAWK
4,51,"121,638",JOHNSON
5,77,"80,784",POTTAWATTAMIE
6,96,"79,840",WOODBURY
7,84,"75,796",STORY
8,30,"66,216",DUBUQUE
9,16,"49,471",CERRO GORDO


In [35]:
best_rcounty_2022 = df.query("year == '2022' & invoice_type == 'RINV'").groupby(['invoice_type'])['county_ID'].value_counts().nlargest(10)

df_best_rcounty_2022 = best_rcounty_2022.to_frame(name='count')

final_df_best_rcounty_2022 = df_best_rcounty_2022.merge(df_county, on=['county_ID'], how = 'left')
final_df_best_rcounty_2022

,county_ID,count,county
0,76,173,POLK
1,56,94,LINN
2,81,64,SCOTT
3,6,57,BLACK HAWK
4,51,50,JOHNSON
5,22,41,CLINTON
6,69,29,MUSCATINE
7,30,28,DUBUQUE
8,96,24,WOODBURY
9,77,22,POTTAWATTAMIE


In [92]:
# Search by County ID

search_invoices_byCountyID = df.query("county_ID == 76").groupby(['invoice_type'])['year'].value_counts()
search_invoices_byCountyID = search_invoices_byCountyID.map('{:,d}'.format).to_frame(name='count')
search_invoices_byCountyID

count
invoice_type year         
INV          2022  501,984
             2021  501,864
             2020  494,816
             2019  440,234
             2018  432,979
RINV         2022      173

### Best Item

In [36]:
best_item_2018 = df.query("year == '2018'").groupby(['invoice_type'])['itemno'].value_counts().nlargest(10)

df_best_item_2018 = best_item_2018.to_frame(name='count')

df_best_item_2018_merge_name = df_best_item_2018.merge(df_item, on=['itemno'], how = 'left')
df_best_item_2018_merge_name_dropcol = df_best_item_2018_merge_name.drop(['item_group', 'pack', 'bottle_volume_ml',
                                                                         'state_bottle_cost', 'state_bottle_retail'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_item_2018_merge_cat = df_best_item_2018_merge_name_dropcol.merge(df_cat, on=['category_ID'], how = 'left')
df_best_item_2018_merge_cat_dropcol = df_best_item_2018_merge_cat.drop(['category_ID'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_item_2018_merge_vendor = df_best_item_2018_merge_cat_dropcol.merge(df_vendor, on=['vendor_ID'], how = 'left')

final_df_best_item_2018 = df_best_item_2018_merge_vendor.drop(['vendor_ID'], axis =1).drop_duplicates().reset_index(drop=True)
final_df_best_item_2018

,itemno,count,im_desc,category,vendor
0,11788,23155,BLACK VELVET,CANADIAN WHISKIES,HEAVEN HILL BRANDS
1,36308,22093,HAWKEYE VODKA,AMERICAN VODKA,LUXCO INC
2,64858,20805,FIREBALL CINNAMON WHISKEY MINI DISPENSER,WHISKEY LIQUEURS,SAZERAC COMPANY INC
3,35918,16025,FIVE O'CLOCK VODKA,AMERICAN VODKA,LAIRD AND COMPANY
4,38176,13805,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC
5,11776,13368,BLACK VELVET,CANADIAN WHISKIES,HEAVEN HILL BRANDS
6,36904,13325,MCCORMICK 80PRF VODKA PET,AMERICAN VODKA,MCCORMICK DISTILLING COMPANY
7,64866,12149,FIREBALL CINNAMON WHISKEY,WHISKEY LIQUEURS,SAZERAC COMPANY INC
8,11774,11615,BLACK VELVET,CANADIAN WHISKIES,HEAVEN HILL BRANDS
9,64864,11536,FIREBALL CINNAMON WHISKEY,WHISKEY LIQUEURS,SAZERAC COMPANY INC


In [37]:
best_item_2019 = df.query("year == '2019'").groupby(['invoice_type'])['itemno'].value_counts().nlargest(10)

df_best_item_2019 = best_item_2019.to_frame(name='count')

df_best_item_2019_merge_name = df_best_item_2019.merge(df_item, on=['itemno'], how = 'left')
df_best_item_2019_merge_name_dropcol = df_best_item_2019_merge_name.drop(['item_group', 'pack', 'bottle_volume_ml',
                                                                         'state_bottle_cost', 'state_bottle_retail'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_item_2019_merge_cat = df_best_item_2019_merge_name_dropcol.merge(df_cat, on=['category_ID'], how = 'left')
df_best_item_2019_merge_cat_dropcol = df_best_item_2019_merge_cat.drop(['category_ID'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_item_2019_merge_vendor = df_best_item_2019_merge_cat_dropcol.merge(df_vendor, on=['vendor_ID'], how = 'left')

final_df_best_item_2019 = df_best_item_2019_merge_vendor.drop(['vendor_ID'], axis =1).drop_duplicates().reset_index(drop=True)
final_df_best_item_2019

,itemno,count,im_desc,category,vendor
0,11788,23713,BLACK VELVET,CANADIAN WHISKIES,HEAVEN HILL BRANDS
1,64858,21992,FIREBALL CINNAMON WHISKEY MINI DISPENSER,WHISKEY LIQUEURS,SAZERAC COMPANY INC
2,36308,21078,HAWKEYE VODKA,AMERICAN VODKA,LUXCO INC
3,38176,16001,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC
4,35918,15511,FIVE O'CLOCK VODKA,AMERICAN VODKA,LAIRD AND COMPANY
5,64870,14417,FIREBALL CINNAMON WHISKEY,WHISKEY LIQUEURS,SAZERAC COMPANY INC
6,36904,14271,MCCORMICK 80PRF VODKA PET,AMERICAN VODKA,MCCORMICK DISTILLING COMPANY
7,11776,13192,BLACK VELVET,CANADIAN WHISKIES,HEAVEN HILL BRANDS
8,38178,12934,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC
9,64864,12260,FIREBALL CINNAMON WHISKEY,WHISKEY LIQUEURS,SAZERAC COMPANY INC


In [38]:
best_item_2020 = df.query("year == '2020'").groupby(['invoice_type'])['itemno'].value_counts().nlargest(10)

df_best_item_2020 = best_item_2020.to_frame(name='count')

df_best_item_2020_merge_name = df_best_item_2020.merge(df_item, on=['itemno'], how = 'left')
df_best_item_2020_merge_name_dropcol = df_best_item_2020_merge_name.drop(['item_group', 'pack', 'bottle_volume_ml',
                                                                         'state_bottle_cost', 'state_bottle_retail'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_item_2020_merge_cat = df_best_item_2020_merge_name_dropcol.merge(df_cat, on=['category_ID'], how = 'left')
df_best_item_2020_merge_cat_dropcol = df_best_item_2020_merge_cat.drop(['category_ID'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_item_2020_merge_vendor = df_best_item_2020_merge_cat_dropcol.merge(df_vendor, on=['vendor_ID'], how = 'left')

final_df_best_item_2020 = df_best_item_2020_merge_vendor.drop(['vendor_ID'], axis =1).drop_duplicates().reset_index(drop=True)
final_df_best_item_2020

,itemno,count,im_desc,category,vendor
0,11788,25342,BLACK VELVET,CANADIAN WHISKIES,HEAVEN HILL BRANDS
1,36308,23458,HAWKEYE VODKA,AMERICAN VODKA,LUXCO INC
2,64858,21285,FIREBALL CINNAMON WHISKEY MINI DISPENSER,WHISKEY LIQUEURS,SAZERAC COMPANY INC
3,38176,18615,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC
4,38178,17026,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC
5,64870,16627,FIREBALL CINNAMON WHISKEY,WHISKEY LIQUEURS,SAZERAC COMPANY INC
6,36904,15787,MCCORMICK 80PRF VODKA PET,AMERICAN VODKA,MCCORMICK DISTILLING COMPANY
7,64864,14722,FIREBALL CINNAMON WHISKEY,WHISKEY LIQUEURS,SAZERAC COMPANY INC
8,35918,13816,FIVE O'CLOCK VODKA,AMERICAN VODKA,LAIRD AND COMPANY
9,48105,13041,HENNESSY VS,IMPORTED BRANDIES,MOET HENNESSY USA


In [39]:
best_item_2021 = df.query("year == '2021'").groupby(['invoice_type'])['itemno'].value_counts().nlargest(10)

df_best_item_2021 = best_item_2021.to_frame(name='count')

df_best_item_2021_merge_name = df_best_item_2021.merge(df_item, on=['itemno'], how = 'left')
df_best_item_2021_merge_name_dropcol = df_best_item_2021_merge_name.drop(['item_group', 'pack', 'bottle_volume_ml',
                                                                         'state_bottle_cost', 'state_bottle_retail'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_item_2021_merge_cat = df_best_item_2021_merge_name_dropcol.merge(df_cat, on=['category_ID'], how = 'left')
df_best_item_2021_merge_cat_dropcol = df_best_item_2021_merge_cat.drop(['category_ID'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_item_2021_merge_vendor = df_best_item_2021_merge_cat_dropcol.merge(df_vendor, on=['vendor_ID'], how = 'left')

final_df_best_item_2021 = df_best_item_2021_merge_vendor.drop(['vendor_ID'], axis =1).drop_duplicates().reset_index(drop=True)
final_df_best_item_2021

,itemno,count,im_desc,category,vendor
0,11788,25424,BLACK VELVET,CANADIAN WHISKIES,HEAVEN HILL BRANDS
1,36308,24467,HAWKEYE VODKA,AMERICAN VODKA,LUXCO INC
2,38176,20168,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC
3,64870,17276,FIREBALL CINNAMON WHISKEY,WHISKEY LIQUEURS,SAZERAC COMPANY INC
4,64858,17168,FIREBALL CINNAMON WHISKEY MINI DISPENSER,WHISKEY LIQUEURS,SAZERAC COMPANY INC
5,38178,17057,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC
6,36904,16993,MCCORMICK 80PRF VODKA PET,AMERICAN VODKA,MCCORMICK DISTILLING COMPANY
7,64864,15908,FIREBALL CINNAMON WHISKEY,WHISKEY LIQUEURS,SAZERAC COMPANY INC
8,10807,14737,CROWN ROYAL REGAL APPLE,CANADIAN WHISKIES,DIAGEO AMERICAS
9,11774,13482,BLACK VELVET,CANADIAN WHISKIES,HEAVEN HILL BRANDS


In [40]:
best_item_2022 = df.query("year == '2022' & invoice_type == 'INV'").groupby(['invoice_type'])['itemno'].value_counts().nlargest(10)

df_best_item_2022 = best_item_2022.to_frame(name='count')

df_best_item_2022_merge_name = df_best_item_2022.merge(df_item, on=['itemno'], how = 'left')
df_best_item_2022_merge_name_dropcol = df_best_item_2022_merge_name.drop(['item_group', 'pack', 'bottle_volume_ml',
                                                                         'state_bottle_cost', 'state_bottle_retail'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_item_2022_merge_cat = df_best_item_2022_merge_name_dropcol.merge(df_cat, on=['category_ID'], how = 'left')
df_best_item_2022_merge_cat_dropcol = df_best_item_2022_merge_cat.drop(['category_ID'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_item_2022_merge_vendor = df_best_item_2022_merge_cat_dropcol.merge(df_vendor, on=['vendor_ID'], how = 'left')

final_df_best_item_2022 = df_best_item_2022_merge_vendor.drop(['vendor_ID'], axis =1).drop_duplicates().reset_index(drop=True)
final_df_best_item_2022

,itemno,count,im_desc,category,vendor
0,11788,25817,BLACK VELVET,CANADIAN WHISKIES,HEAVEN HILL BRANDS
1,36308,24206,HAWKEYE VODKA,AMERICAN VODKA,LUXCO INC
2,38176,20625,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC
3,65013,19169,FIREBALL CINNAMON WHISKEY MINI SLEEVE,WHISKEY LIQUEURS,SAZERAC COMPANY INC
4,65013,19169,FIREBALL CINNAMON WHISKEY 50ML SLEEVE,WHISKEY LIQUEURS,SAZERAC COMPANY INC
5,38178,18211,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC
6,64870,18083,FIREBALL CINNAMON WHISKEY,WHISKEY LIQUEURS,SAZERAC COMPANY INC
7,36904,17660,MCCORMICK 80PRF VODKA PET,AMERICAN VODKA,MCCORMICK DISTILLING COMPANY
8,10802,15501,CROWN ROYAL PEACH,TEMPORARY & SPECIALTY PACKAGES,DIAGEO AMERICAS
9,64864,15328,FIREBALL CINNAMON WHISKEY,WHISKEY LIQUEURS,SAZERAC COMPANY INC


In [41]:
best_ritem_2022 = df.query("year == '2022' & invoice_type == 'RINV'").groupby(['invoice_type'])['itemno'].value_counts().nlargest(10)

df_best_ritem_2022 = best_ritem_2022.to_frame(name='count')

df_best_ritem_2022_merge_name = df_best_ritem_2022.merge(df_item, on=['itemno'], how = 'left')
df_best_ritem_2022_merge_name_dropcol = df_best_ritem_2022_merge_name.drop(['item_group', 'pack', 'bottle_volume_ml',
                                                                         'state_bottle_cost', 'state_bottle_retail'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_ritem_2022_merge_cat = df_best_ritem_2022_merge_name_dropcol.merge(df_cat, on=['category_ID'], how = 'left')
df_best_ritem_2022_merge_cat_dropcol = df_best_ritem_2022_merge_cat.drop(['category_ID'], 
                                                                         axis =1).drop_duplicates().reset_index(drop=True)

df_best_ritem_2022_merge_vendor = df_best_ritem_2022_merge_cat_dropcol.merge(df_vendor, on=['vendor_ID'], how = 'left')

final_df_best_ritem_2022 = df_best_ritem_2022_merge_vendor.drop(['vendor_ID'], axis =1).drop_duplicates().reset_index(drop=True)
final_df_best_ritem_2022

,itemno,count,im_desc,category,vendor
0,65013,23,FIREBALL CINNAMON WHISKEY MINI SLEEVE,WHISKEY LIQUEURS,SAZERAC COMPANY INC
1,65013,23,FIREBALL CINNAMON WHISKEY 50ML SLEEVE,WHISKEY LIQUEURS,SAZERAC COMPANY INC
2,38176,17,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC
3,64870,14,FIREBALL CINNAMON WHISKEY,WHISKEY LIQUEURS,SAZERAC COMPANY INC
4,36904,12,MCCORMICK 80PRF VODKA PET,AMERICAN VODKA,MCCORMICK DISTILLING COMPANY
5,11788,11,BLACK VELVET,CANADIAN WHISKIES,HEAVEN HILL BRANDS
6,38177,10,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC
7,43337,8,CAPTAIN MORGAN ORIGINAL SPICED,SPICED RUM,DIAGEO AMERICAS
8,36304,7,HAWKEYE VODKA,AMERICAN VODKA,LUXCO INC
9,38178,7,TITOS HANDMADE VODKA,AMERICAN VODKA,FIFTH GENERATION INC


In [93]:
# Search by Item ID

search_invoices_byItemID = df.query("itemno == 11788").groupby(['invoice_type'])['year'].value_counts()
search_invoices_byItemID = search_invoices_byItemID.map('{:,d}'.format).to_frame(name='count')
search_invoices_byItemID

count
invoice_type year        
INV          2022  25,817
             2021  25,424
             2020  25,342
             2019  23,713
             2018  23,155
RINV         2022      11

### Best Category

In [42]:
best_cat_2018 = df.query("year == '2018'").groupby(['invoice_type'])['category_ID'].value_counts().nlargest(10)

df_best_cat_2018 = best_cat_2018.to_frame(name='count')

final_df_best_cat_2018 = df_best_cat_2018.merge(df_cat, on=['category_ID'], how = 'left')
final_df_best_cat_2018

,category_ID,count,category
0,9.0,357059,AMERICAN VODKA
1,13.0,231209,CANADIAN WHISKIES
2,44.0,149571,STRAIGHT BOURBON WHISKIES
3,6.0,118187,AMERICAN FLAVORED VODKA
4,43.0,117934,SPICED RUM
5,49.0,108737,WHISKEY LIQUEURS
6,11.0,96865,BLENDED WHISKIES
7,30.0,84513,IMPORTED VODKA
8,2.0,80060,AMERICAN BRANDIES
9,7.0,75998,AMERICAN SCHNAPPS


In [43]:
best_cat_2019 = df.query("year == '2019'").groupby(['invoice_type'])['category_ID'].value_counts().nlargest(10)

df_best_cat_2019 = best_cat_2019.to_frame(name='count')

final_df_best_cat_2019 = df_best_cat_2019.merge(df_cat, on=['category_ID'], how = 'left')
final_df_best_cat_2019

,category_ID,count,category
0,9.0,363240,AMERICAN VODKA
1,13.0,236103,CANADIAN WHISKIES
2,44.0,152965,STRAIGHT BOURBON WHISKIES
3,49.0,121128,WHISKEY LIQUEURS
4,6.0,115060,AMERICAN FLAVORED VODKA
5,43.0,114533,SPICED RUM
6,11.0,100229,BLENDED WHISKIES
7,30.0,79791,IMPORTED VODKA
8,7.0,77745,AMERICAN SCHNAPPS
9,2.0,75531,AMERICAN BRANDIES


In [44]:
best_cat_2020 = df.query("year == '2020'").groupby(['invoice_type'])['category_ID'].value_counts().nlargest(10)

df_best_cat_2020 = best_cat_2020.to_frame(name='count')

final_df_best_cat_2020 = df_best_cat_2020.merge(df_cat, on=['category_ID'], how = 'left')
final_df_best_cat_2020

,category_ID,count,category
0,9.0,382724,AMERICAN VODKA
1,13.0,247546,CANADIAN WHISKIES
2,44.0,177570,STRAIGHT BOURBON WHISKIES
3,49.0,139678,WHISKEY LIQUEURS
4,6.0,123153,AMERICAN FLAVORED VODKA
5,43.0,117249,SPICED RUM
6,11.0,115900,BLENDED WHISKIES
7,14.0,90045,COCKTAILS / RTD
8,30.0,82193,IMPORTED VODKA
9,0.0,81528,100% AGAVE TEQUILA


In [45]:
best_cat_2021 = df.query("year == '2021'").groupby(['invoice_type'])['category_ID'].value_counts().nlargest(10)

df_best_cat_2021 = best_cat_2021.to_frame(name='count')

final_df_best_cat_2021 = df_best_cat_2021.merge(df_cat, on=['category_ID'], how = 'left')
final_df_best_cat_2021

,category_ID,count,category
0,9.0,389647,AMERICAN VODKA
1,13.0,250665,CANADIAN WHISKIES
2,44.0,188848,STRAIGHT BOURBON WHISKIES
3,49.0,144631,WHISKEY LIQUEURS
4,6.0,117915,AMERICAN FLAVORED VODKA
5,43.0,112919,SPICED RUM
6,11.0,112561,BLENDED WHISKIES
7,0.0,89024,100% AGAVE TEQUILA
8,14.0,84374,COCKTAILS / RTD
9,7.0,83797,AMERICAN SCHNAPPS


In [46]:
best_cat_2022 = df.query("year == '2022' & invoice_type == 'INV'").groupby(['invoice_type'])['category_ID'].value_counts().nlargest(10)

df_best_cat_2022 = best_cat_2022.to_frame(name='count')

final_df_best_cat_2022 = df_best_cat_2022.merge(df_cat, on=['category_ID'], how = 'left')
final_df_best_cat_2022

,category_ID,count,category
0,9.0,392293,AMERICAN VODKA
1,13.0,226783,CANADIAN WHISKIES
2,44.0,190975,STRAIGHT BOURBON WHISKIES
3,49.0,152419,WHISKEY LIQUEURS
4,6.0,125588,AMERICAN FLAVORED VODKA
5,43.0,105654,SPICED RUM
6,0.0,103748,100% AGAVE TEQUILA
7,11.0,97299,BLENDED WHISKIES
8,7.0,83686,AMERICAN SCHNAPPS
9,14.0,77073,COCKTAILS / RTD


In [47]:
best_rcat_2022 = df.query("year == '2022' & invoice_type == 'RINV'").groupby(['invoice_type'])['category_ID'].value_counts().nlargest(10)

df_best_rcat_2022 = best_rcat_2022.to_frame(name='count')

final_df_best_rcat_2022 = df_best_rcat_2022.merge(df_cat, on=['category_ID'], how = 'left')
final_df_best_rcat_2022

,category_ID,count,category
0,9.0,170,AMERICAN VODKA
1,13.0,84,CANADIAN WHISKIES
2,49.0,80,WHISKEY LIQUEURS
3,44.0,78,STRAIGHT BOURBON WHISKIES
4,46.0,62,TEMPORARY & SPECIALTY PACKAGES
5,43.0,43,SPICED RUM
6,0.0,40,100% AGAVE TEQUILA
7,3.0,40,AMERICAN CORDIALS & LIQUEURS
8,11.0,37,BLENDED WHISKIES
9,6.0,32,AMERICAN FLAVORED VODKA


In [94]:
# Search by Category ID

search_invoices_byCatID = df.query("category_ID == 9.0").groupby(['invoice_type'])['year'].value_counts()
search_invoices_byCatID = search_invoices_byCatID.map('{:,d}'.format).to_frame(name='count')
search_invoices_byCatID

count
invoice_type year         
INV          2022  392,293
             2021  389,647
             2020  382,724
             2019  363,240
             2018  357,059
RINV         2022      170

### Best Vendor

In [48]:
best_vendor_2018 = df.query("year == '2018'").groupby(['invoice_type'])['vendor_ID'].value_counts().nlargest(10)

df_best_vendor_2018 = best_vendor_2018.to_frame(name='count')

final_df_best_vendor_2018 = df_best_vendor_2018.merge(df_vendor, on=['vendor_ID'], how = 'left')
final_df_best_vendor_2018

,vendor_ID,count,vendor
0,81.0,383232,DIAGEO AMERICAS
1,266.0,348482,SAZERAC COMPANY INC
2,157.0,204675,JIM BEAM BRANDS
3,177.0,197957,LUXCO INC
4,132.0,191961,HEAVEN HILL BRANDS
5,232.0,139538,PERNOD RICARD USA
6,15.0,110843,BACARDI USA INC
7,98.0,94417,E AND J GALLO WINERY
8,241.0,87527,PROXIMO
9,37.0,76348,BROWN FORMAN CORPORATION


In [49]:
best_vendor_2019 = df.query("year == '2019'").groupby(['invoice_type'])['vendor_ID'].value_counts().nlargest(10)

df_best_vendor_2019 = best_vendor_2019.to_frame(name='count')

final_df_best_vendor_2019 = df_best_vendor_2019.merge(df_vendor, on=['vendor_ID'], how = 'left')
final_df_best_vendor_2019

,vendor_ID,count,vendor
0,81.0,387117,DIAGEO AMERICAS
1,266.0,351712,SAZERAC COMPANY INC
2,157.0,203398,JIM BEAM BRANDS
3,132.0,193020,HEAVEN HILL BRANDS
4,177.0,189004,LUXCO INC
5,232.0,136982,PERNOD RICARD USA
6,15.0,111328,BACARDI USA INC
7,241.0,92250,PROXIMO
8,98.0,91926,E AND J GALLO WINERY
9,37.0,77755,BROWN FORMAN CORPORATION


In [50]:
best_vendor_2020 = df.query("year == '2020'").groupby(['invoice_type'])['vendor_ID'].value_counts().nlargest(10)

df_best_vendor_2020 = best_vendor_2020.to_frame(name='count')

final_df_best_vendor_2020 = df_best_vendor_2020.merge(df_vendor, on=['vendor_ID'], how = 'left')
final_df_best_vendor_2020

,vendor_ID,count,vendor
0,81.0,405707,DIAGEO AMERICAS
1,266.0,397834,SAZERAC COMPANY INC
2,157.0,217847,JIM BEAM BRANDS
3,132.0,202822,HEAVEN HILL BRANDS
4,177.0,190022,LUXCO INC
5,232.0,150894,PERNOD RICARD USA
6,241.0,121404,PROXIMO
7,15.0,117303,BACARDI USA INC
8,98.0,107287,E AND J GALLO WINERY
9,37.0,86845,BROWN FORMAN CORPORATION


In [51]:
best_vendor_2021 = df.query("year == '2021'").groupby(['invoice_type'])['vendor_ID'].value_counts().nlargest(10)

df_best_vendor_2021 = best_vendor_2021.to_frame(name='count')

final_df_best_vendor_2021 = df_best_vendor_2021.merge(df_vendor, on=['vendor_ID'], how = 'left')
final_df_best_vendor_2021

,vendor_ID,count,vendor
0,81.0,410212,DIAGEO AMERICAS
1,266.0,403192,SAZERAC COMPANY INC
2,157.0,223435,JIM BEAM BRANDS
3,132.0,203282,HEAVEN HILL BRANDS
4,177.0,182070,LUXCO INC
5,232.0,138842,PERNOD RICARD USA
6,15.0,112654,BACARDI USA INC
7,241.0,110925,PROXIMO
8,98.0,107807,E AND J GALLO WINERY
9,37.0,85097,BROWN FORMAN CORPORATION


In [52]:
best_vendor_2022 = df.query("year == '2022' & invoice_type == 'INV'").groupby(['invoice_type'])['vendor_ID'].value_counts().nlargest(10)

df_best_vendor_2022 = best_vendor_2022.to_frame(name='count')

final_df_best_vendor_2022 = df_best_vendor_2022.merge(df_vendor, on=['vendor_ID'], how = 'left')
final_df_best_vendor_2022

,vendor_ID,count,vendor
0,266.0,415078,SAZERAC COMPANY INC
1,81.0,394612,DIAGEO AMERICAS
2,157.0,215620,JIM BEAM BRANDS
3,132.0,199783,HEAVEN HILL BRANDS
4,177.0,167886,LUXCO INC
5,232.0,120451,PERNOD RICARD USA
6,241.0,108448,PROXIMO
7,15.0,108435,BACARDI USA INC
8,98.0,101686,E AND J GALLO WINERY
9,37.0,77332,BROWN FORMAN CORPORATION


In [53]:
best_rvendor_2022 = df.query("year == '2022' & invoice_type == 'RINV'").groupby(['invoice_type'])['vendor_ID'].value_counts().nlargest(10)

df_best_rvendor_2022 = best_rvendor_2022.to_frame(name='count')

final_df_best_rvendor_2022 = df_best_rvendor_2022.merge(df_vendor, on=['vendor_ID'], how = 'left')
final_df_best_rvendor_2022

,vendor_ID,count,vendor
0,266.0,214,SAZERAC COMPANY INC
1,81.0,156,DIAGEO AMERICAS
2,132.0,76,HEAVEN HILL BRANDS
3,157.0,61,JIM BEAM BRANDS
4,177.0,60,LUXCO INC
5,232.0,53,PERNOD RICARD USA
6,15.0,43,BACARDI USA INC
7,106.0,41,FIFTH GENERATION INC
8,37.0,39,BROWN FORMAN CORPORATION
9,241.0,28,PROXIMO


In [95]:
# Search by Vendor ID

search_invoices_byVendorID = df.query("vendor_ID == 266.0").groupby(['invoice_type'])['year'].value_counts()
search_invoices_byVendorID = search_invoices_byVendorID.map('{:,d}'.format).to_frame(name='count')
search_invoices_byVendorID

count
invoice_type year         
INV          2022  415,078
             2021  403,192
             2020  397,834
             2019  351,712
             2018  348,482
RINV         2022      214